In [1]:
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch 
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from scipy.stats import binom
from statsmodels.stats.contingency_tables import mcnemar
import scipy.stats
import scipy.stats as st

# from KNNCT.ipynb import rightWrong_DT, rightWrong_KNN

In [ ]:
df = pd.read_csv('armdatadf.csv')
df['setup'] = df['setup'].astype(int)
X = df[["x","y","z"]]
Y = df["setup"]
facit = Y[::100].values
X.x = (X.x-X.x.mean()) / X.x.std()
X.y = (X.y-X.y.mean()) / X.y.std()
X.z = (X.z-X.z.mean()) / X.z.std()
XM = np.empty([300])

for i in range(1600):
    XM = np.vstack([XM,X[["x","y","z"]][i*100:(i+1)*100].values.ravel()])

XM = np.delete(XM,0,0)

print(XM.shape)

XM_XY  = np.empty([200])
XM_XZ  = np.empty([200])
XM_YZ  = np.empty([200])

for i in range(1600):
    XM_XY = np.vstack([XM_XY,X[["x","y"]][i*100:(i+1)*100].values.ravel()])
    XM_XZ = np.vstack([XM_XZ,X[["x","z"]][i*100:(i+1)*100].values.ravel()])
    XM_YZ = np.vstack([XM_YZ,X[["y","z"]][i*100:(i+1)*100].values.ravel()])
XM_XY = np.delete(XM_XY,0,0)
XM_XZ = np.delete(XM_XZ,0,0)
XM_YZ = np.delete(XM_YZ,0,0)



/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
(1600, 300)


In [ ]:
###################################################################################################################
def ANNTrain(X_train, y_train, h=25, Batch_size=25, number_of_training_iterations_ANN=4000):    
    #hyperparameters
    n_inputs = 300
    n_outputs = 16
    #Single hidden layer neural network with h hidden neurons
    model = torch.nn.Sequential(
        torch.nn.Linear(n_inputs, h),
        torch.nn.Tanh(),
        torch.nn.Linear(h, n_outputs),
        torch.nn.Softmax()
        )

    #defining loss type
    loss_fn = torch.nn.MSELoss(reduction='sum') 
    X_train = torch.from_numpy(X_train).to(torch.float32)
    y_train = torch.from_numpy(y_train).to(torch.float32)
    learning_rate = 0.005


    for t in range(number_of_training_iterations_ANN):
        indexes = np.random.randint(1440, size=Batch_size) #for every person in the X_train dataset, any setup and any repetition. 
        x = torch.tensor(X_train[indexes]).to(torch.float32)
        
        y = torch.tensor(y_train[indexes])
        y = y.to(torch.int64).t().squeeze()
        y = F.one_hot(y, num_classes=16).to(torch.float32)
        y_pred = model(x)
        # print(y_pred.shape)
        loss = loss_fn(y_pred, y)
        
        #delete the following after model is tested
        #if t % 100 == 99:
            #print(loss)

        model.zero_grad()
        #computing gradients
        loss.backward()

        #update parameters with the computed gradients
        with torch.no_grad():
            for param in model.parameters():
                param -= learning_rate * param.grad
    return model

def ANNTest(X_test, y_test, model):
    #y_test = torch.from_numpy(y_test).to(torch.float32) 
    
    # y = torch.tensor(y_test)
    # y = y.to(torch.int64).t().squeeze()

    errors = 0
    y = torch.tensor(y_test)
    y = F.one_hot(y, num_classes=16).to(torch.float32)

    x = torch.from_numpy(X_test).to(torch.float32)
    
    rightWrongPerson = np.zeros(10*16)
    for i in range(16*10):
        with torch.no_grad():
            inp = x[i]
            output = model(inp)

            predIndex = torch.argmax(output)
            
            prediction = torch.zeros([16])
            prediction[predIndex] = 1

            middleStep = y[i]-prediction
            errors += torch.max(middleStep).item()

            if (torch.max(middleStep).item()==0):
                rightWrongPerson[i] = 1

    return rightWrongPerson

In [ ]:
def ANNTrain2Var(X_train, y_train, h=25, Batch_size=25, number_of_training_iterations_ANN=4000):    
    #hyperparameters
    n_inputs = 200
    n_outputs = 16
    #Single hidden layer neural network with h hidden neurons
    model = torch.nn.Sequential(
        torch.nn.Linear(n_inputs, h),
        torch.nn.Tanh(),
        torch.nn.Linear(h, n_outputs),
        torch.nn.Softmax()
        )

    #defining loss type
    loss_fn = torch.nn.MSELoss(reduction='sum') 
    X_train = torch.from_numpy(X_train).to(torch.float32)
    y_train = torch.from_numpy(y_train).to(torch.float32)
    learning_rate = 0.005


    for t in range(number_of_training_iterations_ANN):
        indexes = np.random.randint(1440, size=Batch_size) #for every person in the X_train dataset, any setup and any repetition. 
        x = torch.tensor(X_train[indexes]).to(torch.float32)
        
        y = torch.tensor(y_train[indexes])
        y = y.to(torch.int64).t().squeeze()
        y = F.one_hot(y, num_classes=16).to(torch.float32)
        y_pred = model(x)
        # print(y_pred.shape)
        loss = loss_fn(y_pred, y)
        
        #delete the following after model is tested
        #if t % 100 == 99:
            #print(loss)

        model.zero_grad()
        #computing gradients
        loss.backward()

        #update parameters with the computed gradients
        with torch.no_grad():
            for param in model.parameters():
                param -= learning_rate * param.grad
    return model

def ANNTest2Var(X_test, y_test, model):
    #y_test = torch.from_numpy(y_test).to(torch.float32) 
    
    # y = torch.tensor(y_test)
    # y = y.to(torch.int64).t().squeeze()

    errors = 0
    y = torch.tensor(y_test)
    y = F.one_hot(y, num_classes=16).to(torch.float32)

    x = torch.from_numpy(X_test).to(torch.float32)
    
    rightWrongPerson = np.zeros(10*16)
    for i in range(16*10):
        with torch.no_grad():
            inp = x[i]
            output = model(inp)

            predIndex = torch.argmax(output)
            
            prediction = torch.zeros([16])
            prediction[predIndex] = 1

            middleStep = y[i]-prediction
            errors += torch.max(middleStep).item()

            if (torch.max(middleStep).item()==0):
                rightWrongPerson[i] = 1

    return rightWrongPerson




In [ ]:
#######################################################################################

rightWrongXYZ = np.zeros((10,10*16))
rightWrongXY  = np.zeros((10,10*16))
rightWrongXZ  = np.zeros((10,10*16))
rightWrongYZ  = np.zeros((10,10*16))

for i in range(10):
    test_IDS = list(range(160*i,160*(i+1)))

    y_test   = facit[test_IDS]
    y_train  = np.delete(facit,test_IDS)
    y_train  = y_train-1
    y_test   = y_test-1

    #Training and testing site
    X_test   = XM[test_IDS]
    X_train  = np.delete(XM, test_IDS, axis = 0)
    model = ANNTrain(X_train,y_train)
    rightWrongXYZ[i] = ANNTest(X_test, y_test, model)
    
    X_test   = XM_XY[test_IDS]
    X_train  = np.delete(XM_XY, test_IDS, axis = 0)
    model = ANNTrain2Var(X_train,y_train)
    rightWrongXY[i] = ANNTest2Var(X_test, y_test, model)

    X_test   = XM_XZ[test_IDS]
    X_train  = np.delete(XM_XZ, test_IDS, axis = 0)
    model = ANNTrain2Var(X_train,y_train)
    rightWrongXZ[i] = ANNTest2Var(X_test, y_test, model)

    X_test   = XM_YZ[test_IDS]
    X_train  = np.delete(XM_YZ, test_IDS, axis = 0)
    model = ANNTrain2Var(X_train,y_train)
    rightWrongYZ[i] = ANNTest2Var(X_test, y_test, model)

    #the returned generalization error when testing   (number of correctly classified setups in test)/(length of trajectories in test)

number_of_true  = (rightWrongXYZ == 1).sum() #counts in all dimensions
number_of_wrong = (rightWrongXYZ == 0).sum() #counts in all dimensions
print(number_of_true/(number_of_true+number_of_wrong))
number_of_true  = (rightWrongXY == 1).sum() #counts in all dimensions
number_of_wrong = (rightWrongXY == 0).sum() #counts in all dimensions
print(number_of_true/(number_of_true+number_of_wrong))
number_of_true  = (rightWrongXZ == 1).sum() #counts in all dimensions
number_of_wrong = (rightWrongXZ == 0).sum() #counts in all dimensions
print(number_of_true/(number_of_true+number_of_wrong))
number_of_true  = (rightWrongYZ == 1).sum() #counts in all dimensions
number_of_wrong = (rightWrongYZ == 0).sum() #counts in all dimensions
print(number_of_true/(number_of_true+number_of_wrong))

tensor(20.2749, grad_fn=<MseLossBackward0>)
tensor(17.8261, grad_fn=<MseLossBackward0>)
tensor(16.7220, grad_fn=<MseLossBackward0>)
tensor(18.1534, grad_fn=<MseLossBackward0>)
tensor(18.7981, grad_fn=<MseLossBackward0>)
tensor(17.0758, grad_fn=<MseLossBackward0>)
tensor(18.3397, grad_fn=<MseLossBackward0>)
tensor(16.3785, grad_fn=<MseLossBackward0>)
tensor(16.4514, grad_fn=<MseLossBackward0>)
tensor(16.2803, grad_fn=<MseLossBackward0>)
tensor(17.4049, grad_fn=<MseLossBackward0>)
tensor(16.6649, grad_fn=<MseLossBackward0>)
tensor(14.6231, grad_fn=<MseLossBackward0>)
tensor(18.3008, grad_fn=<MseLossBackward0>)
tensor(15.2463, grad_fn=<MseLossBackward0>)
tensor(15.3943, grad_fn=<MseLossBackward0>)
tensor(16.3608, grad_fn=<MseLossBackward0>)
tensor(15.1797, grad_fn=<MseLossBackward0>)
tensor(16.9877, grad_fn=<MseLossBackward0>)
tensor(14.2626, grad_fn=<MseLossBackward0>)
tensor(14.6033, grad_fn=<MseLossBackward0>)
tensor(21.1520, grad_fn=<MseLossBackward0>)
tensor(12.4714, grad_fn=<MseLoss

In [ ]:


def writing_to_excel():
    df1 = pd.DataFrame(rightWrongXYZ)
    df2 = pd.DataFrame(rightWrongXY)
    df3 = pd.DataFrame(rightWrongXZ)
    df4 = pd.DataFrame(rightWrongYZ)
    filepath = 'rightWrongXYZ.xlsx'
    df1.to_excel(filepath, index=False)
    filepath = 'rightWrongXY.xlsx'
    df2.to_excel(filepath, index=False)
    filepath = 'rightWrongXZ.xlsx'
    df3.to_excel(filepath, index=False)
    filepath = 'rightWrongYZ.xlsx'
    df4.to_excel(filepath, index=False)
    
writing_to_excel()
## save to xlsx file



In [ ]:
df = pd.read_csv('armdatadf.csv')

Y = df[["setup", "person"]]
X = df[["x","y","z", "person"]]

Gen_error = 0
for i in range(1,11):
    X_train = X[(X.person != i)].copy()
    X_train = X_train.drop("person", axis=1)
    X_train['x'] = (X_train['x']-X_train['x'].mean())/X_train['x'].std()
    X_train['y'] = (X_train['y']-X_train['y'].mean())/X_train['y'].std()
    X_train['z'] = (X_train['z']-X_train['z'].mean())/X_train['z'].std()

    X_test  = X[(X.person == i)].copy()
    X_test = X_test.drop("person", axis=1)
    X_test['x'] = (X_test['x']-X_train['x'].mean())/X_train['x'].std()
    X_test['y'] = (X_test['y']-X_train['y'].mean())/X_train['y'].std()
    X_test['z'] = (X_test['z']-X_train['z'].mean())/X_train['z'].std()

    #print(X_test.shape)
    y_train = Y[(Y.person != i)].copy()
    y_train = y_train.drop("person", axis=1)
    y_train = y_train-1
    
    y_test  = Y[(Y.person == i)].copy()
    y_test = y_test.drop("person", axis=1)
    y_test = y_test-1
    #Training site
    model = ANNTrain(X_train,y_train)

    #Testing site
    Gen_error += ANNTest(X_test, y_test, model)
    
    #the returned generalization error when testing   (number of correctly classified setups in test)/(length of trajectories in test)

print(Gen_error/(16*10*10))


In [ ]:
def ANNTrain(X_train, y_train, h=25, Batch_size=25, number_of_training_iterations_ANN=1500):    

    #y_train = (y_train-np.mean(y_train))/np.std(y_train)
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()

    #hyperparameters
    n_inputs = 300
    n_outputs = 16

    #Single hidden layer neural network with h hidden neurons
    model = torch.nn.Sequential(
    torch.nn.Linear(n_inputs, h),
    torch.nn.Tanh(),
    torch.nn.Linear(h, n_outputs),
    torch.nn.Tanh(),
    torch.nn.Linear(h, n_outputs),
    torch.nn.Softmax()
    )

    #defining loss type
    loss_fn = torch.nn.MSELoss(reduction='sum') 
    X_train = torch.from_numpy(X_train).to(torch.float32)
    y_train = torch.from_numpy(y_train).to(torch.float32)
    learning_rate = 0.005


    for t in range(number_of_training_iterations_ANN):
        indexes = np.random.randint(9*16*10, size=Batch_size) #for every person in the X_train dataset, any setup and any repetition. 
        x = torch.zeros([Batch_size,300]) #25 dataelements, follows x1...x100, så y1..y100 så z1...z100
        for i in range(Batch_size):
            x[i,0:100]   = X_train[indexes[i]*100:indexes[i]*100+100,0].squeeze()
            x[i,100:200] = X_train[indexes[i]*100:indexes[i]*100+100,1].squeeze()
            x[i,200:300] = X_train[indexes[i]*100:indexes[i]*100+100,2].squeeze()


        y = torch.zeros([Batch_size,1])
        for i in range(Batch_size):
            y[i] = y_train[indexes[i]*100]
        y = y.to(torch.int64).t().squeeze()
        y = F.one_hot(y, num_classes=16).to(torch.float32)
        
        y_pred = model(x)
        print(y.shape)
        print(y_pred.shape)
        loss = loss_fn(y_pred, y)
        
        #delete the following after model is tested
        if t % 100 == 99:
            print(loss)

        model.zero_grad()
        #computing gradients
        loss.backward()

        #update parameters with the computed gradients
        with torch.no_grad():
            for param in model.parameters():
                param -= learning_rate * param.grad
    return model

def ANNTest(X_test, y_test, model):
    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()
    y_test = torch.from_numpy(y_test).to(torch.float32) #only X_test need be converted to a torch tensor
    errors = 0
    y = torch.zeros([10*16,1])
    for i in range(10*16):
        y[i] = y_test[i*100]

    y = y.to(torch.int64).t().squeeze()
    y = F.one_hot(y, num_classes=16).to(torch.float32)
    X_test = torch.from_numpy(X_test).to(torch.float32)
    
    x = torch.zeros([16*10,300]) #25 dataelements, follows x1...x100, så y1..y100 så z1...z100
    indexes = np.arange(16*10)
    for i in range(16*10):
        x[i,0:100]   = X_test[indexes[i]*100:indexes[i]*100+100,0].squeeze()
        x[i,100:200] = X_test[indexes[i]*100:indexes[i]*100+100,1].squeeze()
        x[i,200:300] = X_test[indexes[i]*100:indexes[i]*100+100,2].squeeze()
    errors = 0.0
    for i in range(16*10):
        with torch.no_grad():
            inp = x[i,0:300]
            output = model(inp)

            predIndex = torch.argmax(output)
            if (predIndex > 15):
                predIndex = 15
                print("nope")
            prediction = torch.zeros([16])
            prediction[predIndex] = 1

            middleStep = y[i]-prediction


            errors += torch.max(middleStep).item()

    return errors



# Statistics section

In [ ]:
!pip install statsmodels==0.13.1

     |████████████████████████████████| 9.8 MB 19.9 MB/s 
     |████████████████████████████████| 233 kB 39.7 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
def mcnemar(y_true, yhatA, yhatB, alpha=0.05):
    # perform McNemars test
    nn = np.zeros((2,2))
    c1 = yhatA - y_true == 0
    c2 = yhatB - y_true == 0

    nn[0,0] = sum(c1 & c2)
    nn[0,1] = sum(c1 & ~c2)
    nn[1,0] = sum(~c1 & c2)
    nn[1,1] = sum(~c1 & ~c2)

    n = sum(nn.flat);
    n12 = nn[0,1]
    n21 = nn[1,0]

    thetahat = (n12-n21)/n
    Etheta = thetahat

    Q = n**2 * (n+1) * (Etheta+1) * (1-Etheta) / ( (n*(n12+n21) - (n12-n21)**2) )

    p = (Etheta + 1)*0.5 * (Q-1)
    q = (1-Etheta)*0.5 * (Q-1)

    

    p = 2*scipy.stats.binom.cdf(min([n12,n21]), n=n12+n21, p=0.5)
    print("Result of McNemars test using alpha=", alpha)
    print("Comparison matrix n")
    print(nn)
    if n12+n21 <= 10:
        print("Warning, n12+n21 is low: n12+n21=",(n12+n21))

    print("Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] = ", CI)
    print("p-value for two-sided test A and B have same accuracy (exact binomial test): p=", p)

    return thetahat, CI, p

### AUGUST AFSNIT:Decision trees and KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

import numpy as np
from collections import Counter

import pandas as pd






def most_frequent(List):
    return max(set(List), key = List.count)

def knn_model_train(X_train, y_train, neighbors): 
    knn_models = [None]*100
    for i in range(100):
        clf = KNeighborsClassifier(n_neighbors=neighbors)
        local_X_train = X_train[X_train.time==i+1].copy().drop(columns = ["time"])
        local_Y_train = Y_train[Y_train.time==i+1].setup
        knn_models[i] = clf.fit(local_X_train, local_Y_train.values.ravel())
    return knn_models

def DT_model_train(X_train, y_train): 
    DT_models = [None]*100
    
    for i in range(100):
        clf = tree.DecisionTreeClassifier()
        local_X_train = X_train[X_train.time==i+1].copy().drop(columns = ["time"])
        local_Y_train = Y_train[Y_train.time==i+1].setup
        DT_models[i] = clf.fit(local_X_train, local_Y_train.values.ravel())
    return DT_models

def model_test(X_test, y_test, models):
    prediction = []
    for j in range(160):
        prediction_vote = []
        for i in range(100):
            m = i
            i += j*100
            y_hat = models[m].predict(X_test[i:i+1]).item(0)
            prediction_vote.append(y_hat)
        prediction.append(most_frequent(prediction_vote))
    facit = y_test["setup"][::100]
    return facit, prediction    






df = pd.read_csv('armdatadf.csv')
#df['setup'] = df['setup'].apply(str)
df['setup'] = df['setup'].astype(int)

Y = df[["setup","person","time"]]
X = df[["x","y","z","person", "time"]]

facit = Y["setup"][::100].values





rightWrong_KNN_all = np.zeros((10,10*16))
rightWrong_KNN_noX = np.zeros((10,10*16))
rightWrong_KNN_noY = np.zeros((10,10*16))
rightWrong_KNN_noZ = np.zeros((10,10*16))

rightWrong_DT_all = np.zeros((10,10*16))
rightWrong_DT_noX = np.zeros((10,10*16))
rightWrong_DT_noY = np.zeros((10,10*16))
rightWrong_DT_noZ = np.zeros((10,10*16))

Gen_error_KNN_all = 0
Gen_error_KNN_noX = 0
Gen_error_KNN_noY = 0
Gen_error_KNN_noZ = 0

Gen_error_DT_all = 0
Gen_error_DT_noX = 0
Gen_error_DT_noY = 0
Gen_error_DT_noZ = 0

n_persons = 10
nn = 1

for i in range(1,n_persons+1):
    #Seperation
    X_train = X[(X.person != i)].copy()
    X_train['x'] = (X_train['x']-X_train['x'].mean())/X_train['x'].std()
    X_train['y'] = (X_train['y']-X_train['y'].mean())/X_train['y'].std()
    X_train['z'] = (X_train['z']-X_train['z'].mean())/X_train['z'].std()

    X_test  = X[(X.person == i)].copy()
    X_test['x'] = (X_test['x']-X_test['x'].mean())/X_test['x'].std()
    X_test['y'] = (X_test['y']-X_test['y'].mean())/X_test['y'].std()
    X_test['z'] = (X_test['z']-X_test['z'].mean())/X_test['z'].std()

    Y_train = Y[(Y.person != i)].copy()
    Y_test  = Y[(Y.person == i)].copy()

    ##################################################################################################################
    #Training site with all 
    knn_models = knn_model_train(X_train[["x","y","z","time"]],Y_train,nn)
    DT_models = DT_model_train(X_train[["x","y","z","time"]],Y_train)

    # Testing site with all
    facit, prediction = model_test(X_test[["x","y","z"]], Y_test, knn_models)
    rightWrong_KNN_all[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_KNN_all += np.mean(facit==prediction)
    print("KNN: Completed person {} with all coordinates with an accuracy of {}".format(i, acc))

    facit, prediction = model_test(X_test[["x","y","z"]], Y_test, DT_models)
    rightWrong_DT_all[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_DT_all += np.mean(facit==prediction)
    print("DT: Completed person {} with all coordinates with an accuracy of {}".format(i, acc))

    ##################################################################################################################
    #Training site with no X 
    knn_models = knn_model_train(X_train[["y","z","time"]],Y_train,nn)
    DT_models = DT_model_train(X_train[["y","z","time"]],Y_train)

    # Testing site with no X
    facit, prediction = model_test(X_test[["y","z"]], Y_test, knn_models)
    rightWrong_KNN_noX[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_KNN_noX += np.mean(facit==prediction)
    print("KNN: Completed person {} with no X with an accuracy of {}".format(i, acc))

    facit, prediction = model_test(X_test[["y","z"]], Y_test, DT_models)
    rightWrong_DT_noX[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_DT_noX += np.mean(facit==prediction)

    print("DT: Completed person {} with no X with an accuracy of {}".format(i, acc))

    ##################################################################################################################
    #Training site with no Z 
    knn_models = knn_model_train(X_train[["y","x","time"]],Y_train,nn)
    DT_models = DT_model_train(X_train[["y","x","time"]],Y_train)

    # Testing site with no Z
    facit, prediction = model_test(X_test[["y","x"]], Y_test, knn_models)
    rightWrong_KNN_noZ[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_KNN_noZ += np.mean(facit==prediction)
    print("KNN: Completed person {} with no Z with an accuracy of {}".format(i, acc))

    facit, prediction = model_test(X_test[["y","x"]], Y_test, DT_models)
    rightWrong_DT_noZ[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_DT_noZ += np.mean(facit==prediction)

    print("DT: Completed person {} with no Z with an accuracy of {}".format(i, acc))

    ##################################################################################################################
    #Training site with no Y 
    knn_models = knn_model_train(X_train[["z","x","time"]],Y_train,nn)
    DT_models = DT_model_train(X_train[["z","x","time"]],Y_train)

    # Testing site with no Y
    facit, prediction = model_test(X_test[["z","x"]], Y_test, knn_models)
    rightWrong_KNN_noY[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_KNN_noY += np.mean(facit==prediction)
    print("KNN: Completed person {} with no Y with an accuracy of {}".format(i, acc))

    facit, prediction = model_test(X_test[["z","x"]], Y_test, DT_models)
    rightWrong_DT_noY[i-1] = (facit==prediction)*1
    acc = np.mean(facit==prediction)
    Gen_error_DT_noY += np.mean(facit==prediction)

    print("DT: Completed person {} with no Y with an accuracy of {}".format(i, acc))
#the returned generalization error when testing   (number of correctly classified setups in test)/(length of trajectories in test)

KNN: Completed person 1 with all coordinates with an accuracy of 0.16875
DT: Completed person 1 with all coordinates with an accuracy of 0.225
KNN: Completed person 1 with no X with an accuracy of 0.11875
DT: Completed person 1 with no X with an accuracy of 0.11875
KNN: Completed person 1 with no Z with an accuracy of 0.1625
DT: Completed person 1 with no Z with an accuracy of 0.14375
KNN: Completed person 1 with no Y with an accuracy of 0.23125
DT: Completed person 1 with no Y with an accuracy of 0.2375
KNN: Completed person 2 with all coordinates with an accuracy of 0.3875
DT: Completed person 2 with all coordinates with an accuracy of 0.41875
KNN: Completed person 2 with no X with an accuracy of 0.30625
DT: Completed person 2 with no X with an accuracy of 0.25
KNN: Completed person 2 with no Z with an accuracy of 0.16875
DT: Completed person 2 with no Z with an accuracy of 0.175
KNN: Completed person 2 with no Y with an accuracy of 0.40625
DT: Completed person 2 with no Y with an ac

NameError: name 'Gen_error_knn_noX' is not defined

In [ ]:
Gen_error_est_knn_noX = 1-(Gen_error_KNN_noX / n_persons)
Gen_error_est_knn_noY = 1-(Gen_error_KNN_noY / n_persons)
Gen_error_est_knn_noZ = 1-(Gen_error_KNN_noZ / n_persons)
Gen_error_est_knn_all = 1-(Gen_error_KNN_all / n_persons)

Gen_error_est_DT_noX  = 1-(Gen_error_DT_noX / n_persons)
Gen_error_est_DT_noY  = 1-(Gen_error_DT_noY / n_persons)
Gen_error_est_DT_noZ  = 1-(Gen_error_DT_noZ / n_persons)
Gen_error_est_DT_all  = 1-(Gen_error_DT_all / n_persons)

print("Completed {} number of neighbours with all coordinates with an estimated generalization error of {}.".format(nn, Gen_error_est_knn_all))
print("Completed {} number of neighbours with no X with an estimated generalization error of {}.".format(nn, Gen_error_est_knn_noX))
print("Completed {} number of neighbours with no Y with an estimated generalization error of {}.".format(nn, Gen_error_est_knn_noY))
print("Completed {} number of neighbours with no Z with an estimated generalization error of {}.".format(nn, Gen_error_est_knn_noZ))

print("Completed Decision Tree with all coordinates an estimated generalization error of {}.".format(Gen_error_est_DT_all))
print("Completed Decision Tree with no X an estimated generalization error of {}.".format(Gen_error_est_DT_noX))
print("Completed Decision Tree with no Y an estimated generalization error of {}.".format(Gen_error_est_DT_noY))
print("Completed Decision Tree with no Z an estimated generalization error of {}.".format(Gen_error_est_DT_noZ))


Completed 1 number of neighbours with all coordinates with an estimated generalization error of 0.6725.
Completed 1 number of neighbours with no X with an estimated generalization error of 0.751875.
Completed 1 number of neighbours with no Y with an estimated generalization error of 0.651875.
Completed 1 number of neighbours with no Z with an estimated generalization error of 0.82125.
Completed Decision Tree with all coordinates an estimated generalization error of 0.6443749999999999.
Completed Decision Tree with no X an estimated generalization error of 0.760625.
Completed Decision Tree with no Y an estimated generalization error of 0.66125.
Completed Decision Tree with no Z an estimated generalization error of 0.816875.


In [ ]:
# df = pd.DataFrame (rightWrong_KNN_all)
# filepath = 'rightWrong_KNN_all.xlsx'
# df.to_excel(filepath, index=False)

# df = pd.DataFrame (rightWrong_KNN_noX)
# filepath = 'rightWrong_KNN_noX.xlsx'
# df.to_excel(filepath, index=False)

# df = pd.DataFrame (rightWrong_KNN_noY)
# filepath = 'rightWrong_KNN_noY.xlsx'
# df.to_excel(filepath, index=False)

# df = pd.DataFrame (rightWrong_KNN_noZ)
# filepath = 'rightWrong_KNN_noZ.xlsx'
# df.to_excel(filepath, index=False)

# df = pd.DataFrame (rightWrong_KNN_noX)
# filepath = 'rightWrong_KNN_noX.xlsx'
# df.to_excel(filepath, index=False)

# df = pd.DataFrame (rightWrong_DT_all)
# filepath = 'rightWrong_DT_all.xlsx'
# df.to_excel(filepath, index=False)

# df = pd.DataFrame (rightWrong_DT_noX)
# filepath = 'rightWrong_DT_noX.xlsx'
# df.to_excel(filepath, index=False)

# df = pd.DataFrame (rightWrong_DT_noY)
# filepath = 'rightWrong_DT_noY.xlsx'
# df.to_excel(filepath, index=False)

# df = pd.DataFrame (rightWrong_DT_noZ)
# filepath = 'rightWrong_DT_noZ.xlsx'
# df.to_excel(filepath, index=False)

## McNemar

In [ ]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set_theme()
from scipy.stats import binom

In [ ]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set_theme()
import scipy.stats
import numpy as np
import scipy.stats as st

models = []
model = pd.read_excel("rightWrong_DT_all.xlsx")
model_DT_ALL= np.array(model)
models.append(model_DT_ALL)

model = pd.read_excel("rightWrong_DT_noX.xlsx")
model_DT_noX= np.array(model)
models.append(model_DT_noX)

model = pd.read_excel("rightWrong_DT_noY.xlsx")
model_DT_noY= np.array(model)
models.append(model_DT_noY)

model = pd.read_excel("rightWrong_DT_noZ.xlsx")
model_DT_noZ= np.array(model)
models.append(model_DT_noZ)

model = pd.read_excel("rightWrong_KNN_all.xlsx")
model_KNN_all= np.array(model)
models.append(model_KNN_all)

model = pd.read_excel("rightWrong_KNN_noX.xlsx")
model_KNN_noX= np.array(model)
models.append(model_KNN_noX)

model = pd.read_excel("rightWrong_KNN_noY.xlsx")
model_KNN_noY= np.array(model)
models.append(model_KNN_noY)

model = pd.read_excel("rightWrong_KNN_noZ.xlsx")
model_KNN_noZ= np.array(model)
models.append(model_KNN_noZ)

n = 1600
alpha = 0.05

def mcnemar(results1, results2, n, alpha):
    n12 = ((results1-results2)==-1).sum()
    n21 = ((results1-results2)==1).sum()


    thetahat = (n12-n21)/n
    Etheta = thetahat

    Q = n**2 * (n+1) * (Etheta+1) * (1-Etheta) / ( (n*(n12+n21) - (n12-n21)**2) )

    p = (Etheta + 1)*0.5 * (Q-1)
    q = (1-Etheta)*0.5 * (Q-1)

    # CI = tuple(lm * 2 - 1 for lm in scipy.stats.beta.interval(1-alpha, a=p, b=q) )

    p = 2*scipy.stats.binom.cdf(min([n12,n21]), n=n12+n21, p=0.5)

    return p

    # print("Result of McNemars test using alpha=", alpha)
    # print("Comparison matrix n")
    # if n12+n21 <= 10:
    #     print("Warning, n12+n21 is low: n12+n21=",(n12+n21))

    # print("Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] = ", CI)
    # print("p-value for two-sided test A and B have same accuracy (exact binomial test): p=", p)

2.2347052835977196e-13

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9cabbf1f-ad60-47b3-af3c-cdf3732148b9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>